<a href="https://colab.research.google.com/github/AdityaLoth/Implementing-YOLOv3-on-google-colab/blob/master/yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: Mounting google drive**


In [0]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files 
# using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/drive')

## **Step 2: Changing your working directory**

First of all create a folder named darknet in your Google Drive at the location '/content/drive/'

Then we'll make this our current working directory.


In [0]:
%cd /drive/My\ Drive/darknet

## **Step 3: Installing cuDNN**

**Check CUDA release version**

In [0]:
!/usr/local/cuda/bin/nvcc --version
!pwd

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
/content


**Install cuDNN according to the current CUDA version**

Now, you need to download cuDNN from Nvidia web site. You'll need to sign up on the site.


*   Download cuDNN from [Nvidia website](https://developer.nvidia.com/cudnn). 
Right now, because we have CUDA 10.0 preinstalled in Colab runtime, you need download [cuDNN v7.5.0.56 for CUDA v10.0 ](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz)- the file is cudnn-10.0-linux-x64-v7.5.0.56.tgz

*   On your Google Drive, create a folder named cuDNN in your folder darknet. Copy the tgz file there



In [0]:
!tar -xzvf cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2



# **Step 4: Installing Darknet**

This notebook works with AlexeyAB Darknet repo, which is based on Pjreddie's Darknet.

Remember that cloning needs to be done only once. Once you clone the project you just have to change the directory on further runs. So comment the first line after first run

In [0]:
!git clone https://github.com/AlexeyAB/darknet/   #Comment this out for future runs
%cd darknet

Now we will make our project.

Before that we will make sure that the followinf flags are set to 1 in our Makefile. Either go to the newly created darknet folder to make changes or just run the following cell.

GPU=1

OPENCV=1

cuDNN=1

When compiling the first time, your output last line has to be something like this:

g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCVpkg-config --cflags opencv-DGPU (...)

In [0]:
!sed -i ‘s/GPU=0/GPU=1/g’ Makefile
!sed -i ‘s/OPENCV=0/OPENCV=1/g’ Makefile
!sed -i ‘s/cuDNN=0/cuDNN=1/g’ Makefile
!make

**Some utility functions to help upload and download**

In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

**Now we will download pretrained weights**

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

# **Step 5: Detecting Objects**

**1. For pictures**

Save your picture to be detected in darknet/data folder and replace 'person.jpg' with your file_name and its extension.

**NOTE:** You can also use the upload method used below to upload you files. But the code for image detection will be the same as in following cell.

In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont_show

imShow('predictions.jpg')

**2. For Videos**

We will use upload method here. But can manually put your videos too.

Now be careful, darknet seems to crash if you provide wrong type of output file extension (non .avi), the syntax should be exact the same
whatever-you-upload.mp4 is file you provide, and whatever-you-want.avi is what you get:

In [0]:
upload()    #The file gets saved in darknet/ folder, you must remember the path
!pwd

Now we run the code for object detection on your uploaded video. This might take a while so be patient.

In [0]:
#Replace the input_file.mp4 with your uploaded file. And the out_file.avi with whatever-you-want.avi 

!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show input_file.mp4 -i 0 -out_filename whatever-you-want.avi

In [0]:
path=  #Mention the path of the output file
download(path)